In [1]:
import numpy as np
import psycopg2
import datetime
import math

In [116]:
npyname = "kddi_18_new_0316"
sttime = "2022-04-01 18:00:00"

In [117]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [75]:
#道路情報に重なるメッシュIDをすべて取得
sql = "SELECT osm_id FROM roadmap"
cur =connection.cursor()
cur.execute(sql)
keylist = []
for row in cur:
    sql = "SELECT a.grid_code"
    sql += " FROM pointdata a , roadmap r "
    sql += " WHERE ST_Distance(a.geom, r.geom) < 0.0013" #0.001125 -> 125m  0.0013 -> 150m
    sql += " AND r.osm_id = " + str(row[0])

    cur2 =connection.cursor()
    cur2.execute(sql)
    childlist = []
    for row2 in cur2:
        childlist.append(row2[0])
    cur2.close()
    keylist.append([str(row[0]), childlist])
cur.close()

In [ ]:
#道路ID, メッシュID
keylist

In [118]:
len(keylist)

3715

In [119]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [ ]:
#データが存在する道路IDごとに交通量を平均化して取得する
cntDt = 0
wks = ""
wknp=""
find = 0
dtstart = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')

cnts = 0
for row in keylist:
    cnts = cnts + 1
    if wks == "":
        wks = row[0]
    elif wks != row[0]:
        if cntDt != 0:
            if find != 0:
                a_1 = np.array([[wks, np.array(lstItems)]])
                wknp = np.append(wknp, a_1, axis=0)
            else:
                find = 1
                a_1 = np.array([[wks, np.array(lstItems)]])
                wknp = a_1
        wks = row[0]
        cntDt = 0

    lstItems = [-1, -1, -1, -1,  -1,-1, -1, -1,  -1, -1,-1, -1,   -1,-1, -1,-1,   -1,-1, -1, -1 ,  -1, -1, -1, -1,
               -1, -1, -1, -1,  -1,-1, -1, -1,  -1, -1,-1, -1,   -1,-1, -1,-1,   -1,-1, -1, -1 ,  -1, -1, -1, -1,
               -1, -1, -1, -1,  -1,-1, -1, -1,  -1, -1,-1, -1,   -1,-1, -1,-1,   -1,-1, -1, -1 ,  -1, -1, -1, -1,
               -1, -1, -1, -1,  -1,-1, -1, -1,  -1, -1,-1, -1,   -1,-1, -1,-1,   -1,-1, -1, -1 ,  -1, -1, -1, -1]

    print(row[0])
    for meshid in row[1]:
        
        sql = "SELECT times,tbran from " + npyname + " WHERE mesh = " + meshid + " order by times"
        cur =connection.cursor()
        cur.execute(sql)
        finds = 0
        
        for row2 in cur:
            dtnow = datetime.datetime.strptime(str(row2[0]), '%Y-%m-%d %H:%M:%S')
            dtdiff = dtnow - dtstart
            finds = math.floor(dtdiff.seconds / 60 / 15)
            if lstItems[finds] == -1:
                lstItems[finds] = row2[1]
            elif lstItems[finds] < row2[1]:
                lstItems[finds] = row2[1]
            cntDt = cntDt + 1
        cur.close()
if cntDt != 0:
    if find != 0:
        a_1 = np.array([[wks, np.array(lstItems)]])
        wknp = np.append(wknp, a_1, axis=0)
    else:
        a_1 = np.array([[wks, np.array(lstItems)]])
        wknp = a_1
print("complete.")


In [121]:
connection.close()

In [ ]:
wknp

In [123]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [124]:
np.savetxt(npyname + '_custom_0320_max.csv',wknp,fmt="%s", delimiter='\t')

In [125]:
sql = "CREATE TABLE " + npyname + "_lastcustomsummary0320_max (times TIMESTAMP NULL DEFAULT NULL,osm_id BIGINT NULL DEFAULT NULL,tbran DOUBLE PRECISION NULL DEFAULT NULL )"
cur =connection.cursor()
cur.execute(sql)
cur.close()
connection.commit()


In [126]:
cur =connection.cursor()

for items in wknp:
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    for params in items[1]:
        sql = "INSERT INTO " + npyname + "_lastcustomsummary0320_max (times, osm_id,tbran) VALUES ('"+ str(dte) + "',"  + str(items[0]) + "," + str(params) + ");"
        cur.execute(sql)
        dte = dte + datetime.timedelta(minutes=15)
cur.close()
connection.commit()

In [127]:
#CreateGeojsonFile